# Define file path

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [6]:
# Replace with Wales Agilysis
agilysis_file_path = "Data/Sustran.gpkg"
# Replace with Wales NCN
ncn_file_path = "Data/National_Cycle_Network_(Public).geojson"
# Replace with Wales Urban and Rural Boundary
urban_rural_boundary_filepath = 'Data/SG_UrbanRural_2020/SG_UrbanRural_2020.shp'

In [10]:
# Read Agilysis data
agilysis = gpd.read_file(agilysis_file_path)

# Transfer the coordinate reference system
agilysis=agilysis.to_crs(epsg=27700) 

In [9]:
# Read the ncn data
ncn = gpd.read_file(ncn_file_path)

# Transfer the coordinate reference system
ncn=ncn.to_crs(epsg=27700) 

In [11]:
# Read urban and rural boundary data
UrbanRural = gpd.read_file(urban_rural_boundary_filepath)

# Transfer the coordinate reference system
UrbanRural=UrbanRural.to_crs(epsg=27700) 

In [17]:
# Preview the appropriate values for selection
print(UrbanRural["UR8Name"].unique())
print(UrbanRural["UR6Name"].unique())
print(UrbanRural["UR3Name"].unique())

# Filter out agilysis roads within rural area
rural_areas = UrbanRural[
    UrbanRural["UR3Name"].isin(["Accessible Rural Areas", "Remote Rural Areas"])]

agilysis_rural = gpd.sjoin(agilysis, rural_areas, how="inner", predicate="intersects")
agilysis_rural = agilysis_rural.drop(columns=["index_right"])

agilysis_rural.head()

['Large Urban Areas' 'Other Urban Areas' 'Accessible Small Towns'
 'Remote Small Towns' 'Very Remote Small Towns' 'Accessible Rural Areas'
 'Remote Rural Areas' 'Very Remote Rural Areas']
['Large Urban Areas' 'Other Urban Areas' 'Accessible Small Towns'
 'Remote Small Towns' 'Accessible Rural Areas' 'Remote Rural Areas']
['Rest of Scotland' 'Accessible Rural Areas' 'Remote Rural Areas']


,OS Link ID,Road Classification,Road Classification Number,Form of Way,Length (km),Trunk Road,Highway Authority Code,Highway Authority Name,Speed Limit (mph),Speed Limit Evidence date,...,UR8Desc,UR6Class,UR6Name,UR6Desc,UR3Class,UR3Name,UR3Desc,UR2Class,UR2Name,UR2Desc
0,osgb4000000004292146,Classified Unnumbered,None,Single Carriageway,2.209182,false,S12000030,Stirling,60,NaT,...,"Areas with a population of less than 3,000 peo...",6,Remote Rural Areas,"Areas with a population of less than 3,000 peo...",3,Remote Rural Areas,"Areas of less than 3,000 people and with a dri...",2,Rural Areas,"Areas of less than 3,000 people"
1,osgb4000000004292268,Unclassified,None,Single Carriageway,1.308137,false,S12000030,Stirling,30,NaT,...,"Areas with a population of less than 3,000 peo...",6,Remote Rural Areas,"Areas with a population of less than 3,000 peo...",3,Remote Rural Areas,"Areas of less than 3,000 people and with a dri...",2,Rural Areas,"Areas of less than 3,000 people"
2,osgb4000000004292271,A Road,A84,Single Carriageway,2.058606,true,S12000030,Stirling,60,2018-09-25,...,"Areas with a population of less than 3,000 peo...",6,Remote Rural Areas,"Areas with a population of less than 3,000 peo...",3,Remote Rural Areas,"Areas of less than 3,000 people and with a dri...",2,Rural Areas,"Areas of less than 3,000 people"
3,osgb4000000004292724,Classified Unnumbered,None,Single Carriageway,0.656623,false,S12000030,Stirling,60,NaT,...,"Areas with a population of less than 3,000 peo...",5,Accessible Rural Areas,"Areas with a population of less than 3,000 peo...",2,Accessible Rural Areas,"Areas of less than 3,000 people and within 30 ...",2,Rural Areas,"Areas of less than 3,000 people"
4,osgb4000000004292780,Unknown,None,Single Carriageway,0.614588,false,S12000030,Stirling,30,NaT,...,"Areas with a population of less than 3,000 peo...",6,Remote Rural Areas,"Areas with a population of less than 3,000 peo...",3,Remote Rural Areas,"Areas of less than 3,000 people and with a dri...",2,Rural Areas,"Areas of less than 3,000 people"


In [20]:
len(agilysis_rural)

9583

## Save the spatial file

In [21]:
# Save the shapefile incase we need to re-run this long process
agilysis_rural.to_file("Data/agilysis_rural.geojson", driver="GeoJSON")

In [22]:
# Read the file to prevent overwriting
agilysis_rural_saved = gpd.read_file('Data/agilysis_rural.geojson')

In [23]:
# What road type are there in the dataset?
agilysis_rural_saved['Road Classification'].unique()

array(['Classified Unnumbered', 'Unclassified', 'A Road', 'Unknown',
       'Not Classified', 'B Road', 'Motorway'], dtype=object)

In [39]:
# Filter rows where road type is not classified. Note there is no minor road in this dataset. 
agilysis_rural_roadtype = agilysis_rural_saved[
    agilysis_rural_saved["Road Classification"].isin([
        "Classified Unnumbered", "Unclassified", "Unknown", "Not Classified"
    ])
]


In [41]:
# Length of the dataset are reduced from 9538 to 6855
len(agilysis_rural_roadtype)

6855

In [42]:
# Filter rows where traffic volume is smaller than 1000vph
agilysis_rural_aadt = agilysis_rural_roadtype[
    agilysis_rural_roadtype["Modelled AADT"] <= 1000
]

In [44]:
# Length reduce from 6855 to 5112
len(agilysis_rural_aadt)

5112

In [49]:
# Filter rows where traffic speed is lower than 33mph
agilysis_rural_speed = agilysis_rural_aadt[
    agilysis_rural_aadt['85th %ile Speed - All Day'] <= 33
]

In [50]:
# Length reduce from 5112 to 3201
len(agilysis_rural_speed)

3201

In [ ]:
# Save the shapefile incase we need to re-run this long process
agilysis_rural_speed.to_file("Data/QuietLaneScotland.geojson", driver="GeoJSON")

In [60]:
import folium

# --- 0) Make WGS84 copies for web maps  ---
roads_wgs = agilysis_rural_speed.to_crs(4326)
boundary_wgs = rural_areas.to_crs(4326)   # or UrbanRural/to your subset

# --- 1) Get a safe map center  ---
# Use total_bounds to avoid heavy unions:
minx, miny, maxx, maxy = roads_wgs.total_bounds
center = [(miny + maxy) / 2, (minx + maxx) / 2]

# --- 2) Make all properties JSON-serializable (convert Timestamps to str) ---
def make_json_safe(gdf):
    gdf = gdf.copy()
    for col in gdf.select_dtypes(include=["datetime64[ns]", "datetime64[ns, UTC]"]).columns:
        gdf[col] = gdf[col].astype(str)
    return gdf

roads_wgs = make_json_safe(roads_wgs)
boundary_wgs = make_json_safe(boundary_wgs)

# drop wide/unused columns to keep the GeoJSON light
keep_cols = ["Modelled AADT", "Road Classification", "geometry"] 
roads_wgs = roads_wgs[[c for c in keep_cols if c in roads_wgs.columns]]

# --- 3) Build the map ---
m = folium.Map(location=center, zoom_start=9, tiles="cartodbpositron")

# Add boundary polygons first (grey outline)
folium.GeoJson(
    boundary_wgs.to_json(),
    name="Rural Areas",
    style_function=lambda f: {
        "fillColor": "lightgrey",   # grey fill
        "color": "white",           # border line color
        "weight": 1,
        "fillOpacity": 0.5,         # transparency of fill
        "opacity": 0.7,
    },
    tooltip=folium.GeoJsonTooltip(fields=["UR3Name"], aliases=["Area Type"])
).add_to(m)

# Add roads, with a simple style by AADT (thin lines)
def road_style(feat):
    aadt = feat["properties"].get("Modelled AADT", None)
    # simple 3-break ramp; tweak as needed
    if aadt is None:
        weight = 1
    elif aadt <= 250:
        weight = 2
    elif aadt <= 500:
        weight = 3
    elif aadt <= 1000:
        weight = 4
    else:
        weight = 5
    return {"color": "#1f77b4", "weight": weight, "opacity": 0.9}

folium.GeoJson(
    roads_wgs.to_json(),
    name="Agilysis Rural Roads",
    style_function=road_style,
    tooltip=folium.GeoJsonTooltip(
        fields=[c for c in ["Road Classification", "Modelled AADT"] if c in roads_wgs.columns],
        aliases=["Classification", "AADT"],
        sticky=True,
    ),
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save("agilysis_rural_quiet_lane_map.html")
